# tnc_edge_bv_excel_parsing

This notebook takes Bureau Veritas's excel spreadsheets as input, and produces three tables as output

```
trips
sets
fish
```

The tables are each a dataframe in this notebook, and exported as CSVs to an S3 bucket for querying with an AWS Cloud DB product


In [11]:
import pandas
import numpy as np
from datetime import datetime, date, time, timezone, timedelta
from dateutil.parser import parse as parse_datetime
# help(np.argwhere)
import re

In [13]:
# dir(pandas)
# help(pandas.read_excel)

fname = '20240308_SAINT PATRICK_FO6_FO7_FO8_FO9_FO10_FO11_FO12_FO13.xlsx'

# boat = 'brancol'
boat = 'stpatrick'

all_sheets = pandas.read_excel(fname, sheet_name=None)

sheet_names = list(filter(lambda k: re.match('^FO \d+', k), all_sheets.keys()))

curr_sheet = all_sheets[sheet_names[1]]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [14]:
def display_full(x):
    pandas.set_option('display.max_rows', 1000)
    pandas.set_option('display.min_rows', 400)
    pandas.set_option('display.max_columns', None)
    pandas.set_option('display.width', 2000)
    pandas.set_option('display.float_format', '{:20,.2f}'.format)
    pandas.set_option('display.max_colwidth', None)
    display(x)
    pandas.reset_option('display.max_rows')
    pandas.reset_option('display.max_columns')
    pandas.reset_option('display.width')
    pandas.reset_option('display.float_format')
    pandas.reset_option('display.max_colwidth')


In [15]:


def findcell(sheet, needle):
    for col_name in list(sheet.keys()):
        try:
            start_idx = sheet[col_name].to_list().index(needle)
            return (col_name, start_idx+1)
        except ValueError:
            pass
    return None

def set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, cell_str, cell_offset):
    cols_index = list(curr_sheet.keys())
    
    if cols_index.index(set_haul_title_cell[0]) + cell_offset[0] >= len(cols_index):
        raise ValueError(f'index offset {cell_offset[0]} out of bounds in sheet {cols_index}')
    
#     print(set_haul_title_cell)
#     print(cell_offset)
#     print(cols_index.index(set_haul_title_cell[0]))
#     print(cols_index[cols_index.index(set_haul_title_cell[0]) + cell_offset[0]])
#     print(curr_sheet[cols_index[cols_index.index(set_haul_title_cell[0]) + cell_offset[0]]])
#     print(curr_sheet[cols_index[cols_index.index(set_haul_title_cell[0]) + cell_offset[0]]][set_haul_title_cell[1] + cell_offset[1]])
                                
    if curr_sheet[cols_index[cols_index.index(set_haul_title_cell[0]) + cell_offset[0]]][set_haul_title_cell[1] + cell_offset[1]] != cell_str:
        raise ValueError(f"can't find {cell_str}")
    return curr_sheet[cols_index[cols_index.index(set_haul_title_cell[0]) + cell_offset[0]]][set_haul_title_cell[1] + cell_offset[1]+1]


def set_haul_grid_fetch_all(curr_sheet, set_haul_title_cell):
    start_date = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'start date', (0, 0))
    try:
        start_time = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'start time (UTC)', (1, 0))
    except:
        start_time = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'start time', (1, 0))
    start_lat = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'latitude', (2, 0))
    start_lon = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'longitude', (3, 0))
    finish_date = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'finish date', (0, 2))
    try:
        finish_time = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'finish time (UTC)', (1, 2))
    except:
        try:
            finish_time = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'finish time (UTC', (1, 2))
        except:
            finish_time = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'finish time', (1, 2))
    finish_lat = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'latitude', (2, 2))
    finish_lon = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'longitude', (3, 2))

    return (start_date, start_time, start_lat, start_lon, finish_date, finish_time, finish_lat, finish_lon)


In [16]:


# trip info


if list(all_sheets.keys())[0] != 'TRIP':
    raise ValueError('first sheet should be trip info')

trip_sheet = all_sheets['TRIP']

if 'Fishing trip' in trip_sheet.keys():
    trip_start_date_cell = ('Fishing trip', 0)
else:
    trip_start_date_cell = findcell(curr_sheet, 'Fishing trip')
    if not setting_cell:
        raise ValueError("no 'Fishing trip' block in sheet")
        

trip_notes = ''
try:
    trip_notes = set_haul_grid_fetch_one(trip_sheet, trip_start_date_cell, 'General notes ', (7,0)) 
except:
    try:
        trip_notes = set_haul_grid_fetch_one(trip_sheet, trip_start_date_cell, 'Notes', (7,0)) 
    except:
        try:
            trip_notes = set_haul_grid_fetch_one(trip_sheet, trip_start_date_cell, 'Notes', (7,1)) 
        except:
            try:
                trip_notes = set_haul_grid_fetch_one(trip_sheet, trip_start_date_cell, 'Note ', (8,0)) 
            except:
                pass
try:
    trip_start_date = set_haul_grid_fetch_one(trip_sheet, trip_start_date_cell, 'Start date', (0,1)).date()
except:
    trip_start_date = set_haul_grid_fetch_one(trip_sheet, trip_start_date_cell, 'Start date', (0,3)).date()

try:
    trip_end_date = set_haul_grid_fetch_one(trip_sheet, trip_start_date_cell, 'Finish date', (2,1)).date()
except:
    trip_end_date = set_haul_grid_fetch_one(trip_sheet, trip_start_date_cell, 'Finish date', (2,3)).date()
trip_id = boat + "_" + str(trip_start_date)

try:
    obsv_name = set_haul_grid_fetch_one(trip_sheet, trip_start_date_cell, "Observer's name", (0,4))
except ValueError as e:
    obsv_name = set_haul_grid_fetch_one(trip_sheet, trip_start_date_cell, "Observer's name", (0,6))
    

trip_data = dict(
trip_id = trip_id,
trip_start_date = trip_start_date,
trip_end_date = trip_end_date,
trip_notes = trip_notes,
obsv_name = obsv_name,
)
trip_data = {k:[v] for k,v in trip_data.items()}
trip_df = pandas.DataFrame(trip_data)
trip_df = trip_df.set_index('trip_id')
# print(trip_df)


# set/haul info

set_number = 0

sets_df = None
fish_df = None

for sheet_name in sheet_names:
    try:
        curr_sheet = all_sheets[sheet_name]

        # look for 'latitude' in this sheet. If found, it's probably a sheet with set/haul data
        fao_code_cell = findcell(curr_sheet, 'FAO code')
        if not fao_code_cell:
            continue
        catchcondition_cell = findcell(curr_sheet, 'catch condition')
        if not catchcondition_cell:
            continue
        discardreason_cell = findcell(curr_sheet, 'reason for discard')
        if not discardreason_cell:
            continue
        if discardreason_cell[1] != catchcondition_cell[1] or catchcondition_cell[1] != discardreason_cell[1]:
            raise ValueError(f"cannot find header row for fish data in sheet {sheet_name}")

        # find other metadata values

        cols_index = list(curr_sheet.keys())
        if 'Setting' in cols_index:
            setting_cell = ('Setting', 0)
        else:
            setting_cell = findcell(curr_sheet, 'Setting')
            if not setting_cell:
                raise ValueError("no 'Setting' block in sheet")

        (set_start_date, set_start_time, set_start_lat ,
            set_start_lon ,set_end_date, set_end_time,set_end_lat ,
            set_end_lon,) = set_haul_grid_fetch_all(curr_sheet, setting_cell)


        if 'Hauling' in cols_index:
            hauling_cell = ('Hauling', 0)
        else:
            hauling_cell = findcell(curr_sheet, 'Hauling')
            if not hauling_cell:
                raise ValueError("no 'Hauling' block in sheet")

        (haul_start_date, haul_start_time, haul_start_lat ,
            haul_start_lon ,haul_end_date, haul_end_time,haul_end_lat ,
            haul_end_lon,) = set_haul_grid_fetch_all(curr_sheet, hauling_cell)

        set_number += 1
        set_id = trip_id + "_set_"+str(set_number).zfill(2)
    #     print(set_id)
        set_row = dict(
            set_id=set_id,
            trip_id=trip_id,
            set_number=set_number,

            set_start_datetime = datetime.combine(set_start_date.date(), set_start_time).replace(tzinfo=timezone.utc),
            set_start_lat = set_start_lat ,
            set_start_lon = set_start_lon ,

            set_end_datetime = datetime.combine(set_end_date.date(), set_end_time).replace(tzinfo=timezone.utc),
            set_end_lat = set_end_lat ,
            set_end_lon = set_end_lon ,

            haul_start_datetime = datetime.combine(haul_start_date.date(), haul_start_time).replace(tzinfo=timezone.utc),
            haul_start_lat = haul_start_lat,
            haul_start_lon = haul_start_lon ,

            haul_end_datetime = datetime.combine(haul_end_date.date(), haul_end_time).replace(tzinfo=timezone.utc),
            haul_end_lat = haul_end_lat,
            haul_end_lon = haul_end_lon
        )
    #         print({k:v for k,v in set_row.items()})
        set_row = {k:[v] for k,v in set_row.items()}

    #         print(set_row)
        set_df = pandas.DataFrame(set_row)
        set_df = set_df.set_index('set_id')
    #         print(set_df)
        if sets_df is None:
            sets_df = set_df
        else:
            sets_df = sets_df.append(set_df)

    #     print(sheet_name, fao_code_cell[1])

        reimport_sheet = pandas.read_excel(fname, sheet_name=sheet_name, skiprows=fao_code_cell[1])
        reimport_sheet = reimport_sheet.loc[:, ~reimport_sheet.columns.str.contains('^Unnamed: ')]
        def replace_catch_hour(catch_hour):
            if type(catch_hour) == str:
    #             print(catch_hour)
                catch_hour = parse_datetime(catch_hour).time()
            haul_datetime = set_row['haul_start_datetime'][0]
            catch_datetime = haul_datetime.replace(hour=catch_hour.hour, minute=catch_hour.minute)
            if haul_datetime - catch_datetime > timedelta(hours=2):
                # the catch_datetime is somehow smaller than the start of the haul
                # this is outside of the haul window
                # this is probably because the haul started just before midnight, and continued to the next day
                catch_datetime += timedelta(days=1)
                end_datetime = set_row['haul_end_datetime'][0]
                if catch_datetime - end_datetime > timedelta(hours=2):
                    # adding a day didn't work, now it's outside of the haul window on the other side
                    raise ValueError(f'catch time {catch_hour} cannot fit between haul times {haul_datetime} - {end_datetime}')
            return catch_datetime

        try:
            reimport_sheet['catch_datetime'] = reimport_sheet['hour'].map(replace_catch_hour)
        except BaseException as e:
            print("error on sheetname", sheet_name)
            raise e

    #     print(reimport_sheet)

        reimport_sheet['set_id'] = set_id
        reimport_sheet.insert(0, 'set_id', reimport_sheet.pop('set_id'))
        reimport_sheet['fish_id'] = reimport_sheet['set_id'] + "_fish_" + pandas.Series(map(lambda i: str(i).zfill(3), reimport_sheet.index.values))
        reimport_sheet = reimport_sheet.set_index('fish_id')

        if fish_df is None:
            fish_df = reimport_sheet
        else:
            fish_df = fish_df.append(reimport_sheet)
    except BaseException as e:
        print('debug - in sheet', sheet_name)
        raise e


display(trip_df)

display(sets_df)

# fish_df.index = pandas.Index(fish_df['fish_id'])

# fish_df.pop('fish_id')

display(fish_df)



,trip_start_date,trip_end_date,trip_notes,obsv_name
trip_id,,,,
stpatrick_2024-03-08,2024-03-08,2024-04-06,,Gael DOUZET


,trip_id,set_number,set_start_datetime,set_start_lat,set_start_lon,set_end_datetime,set_end_lat,set_end_lon,haul_start_datetime,haul_start_lat,haul_start_lon,haul_end_datetime,haul_end_lat,haul_end_lon
set_id,,,,,,,,,,,,,,
stpatrick_2024-03-08_set_01,stpatrick_2024-03-08,1,2024-03-11 11:32:00+00:00,+8.4221784,-88.2238933,2024-03-11 16:40:00+00:00,+8.3222517,-87.8782519,2024-03-12 06:15:00+00:00,+8.2039536,-87.9683316,2024-03-12 14:34:00+00:00,+8.1713183,-88.1606337
stpatrick_2024-03-08_set_02,stpatrick_2024-03-08,2,2024-03-13 17:55:00+00:00,+6.9635806,-87.7458437,2024-03-13 21:50:00+00:00,+6.7690374,-87.4045067,2024-03-14 10:20:00+00:00,+6.6321011,-87.1789986,2024-03-14 13:51:00+00:00,+6.6465680,-87.3375228
stpatrick_2024-03-08_set_03,stpatrick_2024-03-08,3,2024-03-15 18:27:00+00:00,+6.2218952,-89.2712439,2024-03-15 22:50:00+00:00,+5.8553583,-89.5160366,2024-03-16 11:15:00+00:00,+5.8072698,-89.6573541,2024-03-16 16:24:00+00:00,+6.0565391,-89.5363765
stpatrick_2024-03-08_set_04,stpatrick_2024-03-08,4,2024-03-16 19:10:00+00:00,+6.2362759,-87.7458437,2024-03-16 22:44:00+00:00,+6.4959027,-87.4045067,2024-03-17 11:38:00+00:00,+6.5521340,-87.4577272,2024-03-17 15:31:00+00:00,+6.3737745,-89.5126788
stpatrick_2024-03-08_set_05,stpatrick_2024-03-08,5,2024-03-19 11:41:00+00:00,+6.2167170,-89.1539514,2024-03-19 15:55:00+00:00,+6.4995167,-88.9941482,2024-03-20 03:36:00+00:00,+6.3826147,-89.0359938,2024-03-20 16:03:00+00:00,+6.1030053,-89.3638759
stpatrick_2024-03-08_set_06,stpatrick_2024-03-08,6,2024-03-21 15:58:00+00:00,+6.1284698,-89.3967083,2024-03-21 20:48:00+00:00,+6.4048942,-88.6196155,2024-03-22 10:30:00+00:00,+6.3461053,-89.5982785,2024-03-22 16:03:00+00:00,+6.0214554,-89.4294500
stpatrick_2024-03-08_set_07,stpatrick_2024-03-08,7,2024-03-22 16:54:00+00:00,+6.0202763,-89.4251224,2024-03-22 21:52:00+00:00,+6.0763257,-88.7876607,2024-03-23 18:15:00+00:00,+5.9700622,-89.8401508,2024-03-23 22:38:00+00:00,+5.8415494,-89.5665386
stpatrick_2024-03-08_set_08,stpatrick_2024-03-08,8,2024-03-24 15:38:00+00:00,+5.7402276,-89.7331774,2024-03-24 20:20:00+00:00,+5.4317133,-89.6028334,2024-03-25 09:27:00+00:00,+5.4820776,-89.7712457,2024-03-25 17:20:00+00:00,+5.7680910,-90.0746000
stpatrick_2024-03-08_set_09,stpatrick_2024-03-08,9,2024-03-26 12:15:00+00:00,+5.9003888,-90.1501888,2024-03-26 17:14:00+00:00,+5.5889282,-90.1618519,2024-03-27 09:31:00+00:00,+5.6475071,-90.1442468,2024-03-27 15:30:00+00:00,+5.9015676,-90.0941892


,set_id,FAO code,scientific name,hour,latitude,longitude,kind of catch,future,catch condition,fate condition,reason for discard,good pratices ETP,catch_datetime
fish_id,,,,,,,,,,,,,
stpatrick_2024-03-08_set_01_fish_000,stpatrick_2024-03-08_set_01,PLS,Pteroplatytrygon violacea,06:25:00,8.203,-87.976,non commercial species,discarded,alive not injured,alive not injured,non commercial species,Line cut close to the hook,2024-03-12 06:25:00+00:00
stpatrick_2024-03-08_set_01_fish_001,stpatrick_2024-03-08_set_01,FAL,Carcharhinus falciformis,06:29:00,8.203,-87.980,target species,retained,alive not injured,NaN,NaN,NaN,2024-03-12 06:29:00+00:00
stpatrick_2024-03-08_set_01_fish_002,stpatrick_2024-03-08_set_01,FAL,Carcharhinus falciformis,06:54:00,8.204,-88.012,target species,retained,alive not injured,NaN,NaN,NaN,2024-03-12 06:54:00+00:00
stpatrick_2024-03-08_set_01_fish_003,stpatrick_2024-03-08_set_01,FAL,Carcharhinus falciformis,07:02:00,8.196,-88.021,target species,retained,alive not injured,NaN,NaN,NaN,2024-03-12 07:02:00+00:00
stpatrick_2024-03-08_set_01_fish_004,stpatrick_2024-03-08_set_01,PLS,Pteroplatytrygon violacea,07:13:00,8.188,-88.034,non commercial species,discarded,alive not injured,alive not injured,non commercial species,Line cut close to the hook,2024-03-12 07:13:00+00:00
stpatrick_2024-03-08_set_01_fish_005,stpatrick_2024-03-08_set_01,FAL,Carcharhinus falciformis,07:17:00,8.186,-88.037,target species,retained,alive not injured,NaN,NaN,NaN,2024-03-12 07:17:00+00:00
stpatrick_2024-03-08_set_01_fish_006,stpatrick_2024-03-08_set_01,PLS,Pteroplatytrygon violacea,11:56:00,8.140,-88.066,non commercial species,discarded,alive not injured,alive not injured,non commercial species,Line cut close to the hook,2024-03-12 11:56:00+00:00
stpatrick_2024-03-08_set_01_fish_007,stpatrick_2024-03-08_set_01,FAL,Carcharhinus falciformis,12:02:00,8.141,-88.073,target species,retained,alive not injured,NaN,NaN,NaN,2024-03-12 12:02:00+00:00
stpatrick_2024-03-08_set_01_fish_008,stpatrick_2024-03-08_set_01,WAH,Acanthocybium solandri,12:16:00,8.143,-88.082,target species,retained,dead,NaN,NaN,NaN,2024-03-12 12:16:00+00:00


## Export to CSV in S3

the following cells are for AWS Athena usage


In [17]:
import awswrangler as wr
import boto3
boto3.setup_default_session(profile_name='XXXXXXXX')

In [ ]:
# dir(wr.s3)
# wr.s3.list_buckets()
bucket='51-gema-dev-dp-raw'
# wr.s3.list_directories(f's3://{bucket}/tnc_edge/')
# help(wr.s3.to_csv)


print(
    wr.s3.to_csv(trip_df, f's3://{bucket}/tnc_edge/{boat}_v1_bv_trips/{trip_id}.csv'),
    wr.s3.to_csv(sets_df, f's3://{bucket}/tnc_edge/{boat}_v1_bv_sets/{trip_id}.csv'),
    wr.s3.to_csv(fish_df, f's3://{bucket}/tnc_edge/{boat}_v1_bv_fish/{trip_id}.csv')
)
